In [3]:
from Dataset import lerCSV,lerXLSX
import warnings
import sys 
import numpy as np
import pickle
import theanets
from sklearn import preprocessing
from random import shuffle
from sklearn.metrics import precision_score,recall_score
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from math import sqrt
%matplotlib inline

In [25]:
warnings.filterwarnings("ignore")   
matrix = lerXLSX("Banco de Dados - Infarto treinoTeste.xlsx",training=True)

In [26]:
k = 10
lista = list(range(0,len(matrix))) 
shuffle(lista)
a = np.asarray(lista) % k

In [28]:
min_max_scaler = preprocessing.MinMaxScaler()
matrixProcessed = min_max_scaler.fit_transform(matrix[:,:4])
matrixProcessed = preprocessing.normalize(matrix[:,:3], norm='l2')
#matrixProcessed = np.array([[matrixProcessed[y,0],matrixProcessed[y,1],matrixProcessed[y,2],matrixProcessed[y,3]]  for y in range(0,len(matrix[:,0]))])


In [17]:
def KFold(matrix,metric = ["neural network (4,5 tanh,2)",[3, (7, 'tanh'), 2]]):
    k = 8
    
    limit = 3
    
    
    lista = list(range(1,len(matrix))) 
    shuffle(lista)
    a = np.asarray(lista) % k   
    
    scores = np.zeros(k)
    recall = np.zeros(k)
    precision = np.zeros(k)
    average_precision = np.zeros(k)
    
    for fold in range(0,k):
        treinoIndex = np.where(a != fold)[0]
        testeIndex =  np.where(a == fold)[0]        
        
        inputs = np.array([[matrix[y,0],matrix[y,1],matrix[y,2]]  for y in treinoIndex])
        weights = np.array([int(matrix[y,3] == 1)*1+1 for y in treinoIndex],dtype="int32")        
        outputs = np.array([int(matrix[y,3] == 1) for y in treinoIndex],dtype="int32")
            
        model = theanets.Classifier(metric[1],loss='CrossEntropy',weighted=True)  
        model.train([inputs, outputs,weights],algo='sgd',learning_rate=0.01,momentum=0.9)       
        
        testOutputs = [matrix[y,3] for y in testeIndex] 
        testInputs = np.array([matrix[y,:3]  for y in testeIndex])   
        predicts = model.predict(testInputs)
        scores[fold] = sum([int(predicts[x] == testOutputs[x]) for x in range(len(predicts))]) / len(predicts) 
        recall[fold] = recall_score(testOutputs, predicts)
        precision[fold] = precision_score(testOutputs, predicts)        
        
    
    print("Final:"+ metric[0] + " = Score:%2.2e[+/- %2.2e]"%(np.mean(scores),np.std(scores)))
    print("Final:"+ metric[0] + " = recall:%2.2e[+/- %2.2e]"%(np.mean(recall),np.std(recall)))
    print("Final:"+ metric[0] + " = precision:%2.2e[+/- %2.2e]"%(np.mean(precision),np.std(precision)))       
    return np.mean(scores), np.std(scores)

In [18]:
inputOutput = np.array([np.concatenate((matrixProcessed[x,:3],[matrix[x,4] == 1]),axis=0) for x in range(0,len(matrix[:,4]))])

In [19]:
metrics = [ ["neural network (4,5 tanh,2)",[3, (5, 'tanh'), 2]], ["neural network (4,7 tanh,2)",[3, (7, 'tanh'), 2]], ["neural network (4,10 tanh,2)",[3, (10, 'tanh'), 2]], ["neural network (4,15 tanh,2)",[3, (15, 'tanh'), 2]], ["neural network (4,5 tanh,5 tanh,2)",[3,(5, 'tanh'), (5, 'tanh'), 2]]]
for metric in metrics:
    KFold(inputOutput, metric=metric)


Final:neural network (4,5 tanh,2) = Score:5.81e-01[+/- 6.27e-02]
Final:neural network (4,5 tanh,2) = recall:6.88e-01[+/- 7.03e-02]
Final:neural network (4,5 tanh,2) = precision:4.34e-01[+/- 7.63e-02]
Final:neural network (4,7 tanh,2) = Score:5.90e-01[+/- 3.35e-02]
Final:neural network (4,7 tanh,2) = recall:6.71e-01[+/- 9.99e-02]
Final:neural network (4,7 tanh,2) = precision:4.37e-01[+/- 4.63e-02]
Final:neural network (4,10 tanh,2) = Score:5.81e-01[+/- 5.37e-02]
Final:neural network (4,10 tanh,2) = recall:6.49e-01[+/- 1.03e-01]
Final:neural network (4,10 tanh,2) = precision:4.30e-01[+/- 7.70e-02]
Final:neural network (4,15 tanh,2) = Score:5.81e-01[+/- 5.07e-02]
Final:neural network (4,15 tanh,2) = recall:6.58e-01[+/- 7.21e-02]
Final:neural network (4,15 tanh,2) = precision:4.27e-01[+/- 5.28e-02]
Final:neural network (4,5 tanh,5 tanh,2) = Score:5.64e-01[+/- 4.44e-02]
Final:neural network (4,5 tanh,5 tanh,2) = recall:7.23e-01[+/- 1.08e-01]
Final:neural network (4,5 tanh,5 tanh,2) = precis